In [10]:
import os
import pandas as pd
import numpy as np
from scipy.spatial.distance import directed_hausdorff
import pydicom as dcm

# Define patient file directory
directory = r'C:\Users\Hiran Gunawardana\Desktop\KYS Project\NEW'

# Import organ models data
folder_path = r'C:\Users\Hiran Gunawardana\Desktop\KYS Project\Organ Models' 

# Create a dictionary to store the model organs dataframes
model_organs = {}
for file in os.scandir(folder_path):
    if file.name.endswith('.csv') and file.is_file():
        file_name = os.path.splitext(file.name)[0]  # Extract the file name without extension
        df = pd.read_csv(file.path)  # Read CSV into dataframe
        model_organs[file_name] = df  # Store dataframe in the dictionary with file name as key

result_dict = {}

for foldername in os.listdir(directory):
    folderpath = os.path.join(directory, foldername)
    if os.path.isdir(folderpath):
        for filename in os.listdir(folderpath):
            filepath = os.path.join(folderpath, filename)
            if os.path.isfile(filepath) and filepath.endswith('.dcm'):
                dcm_file = dcm.dcmread(filepath)
                # Proceeding only if the DICOM file is from the RTSTRUCT modality
                if dcm_file.Modality == "RTSTRUCT":
                    
                    contours = dcm_file.ROIContourSequence
                    organs = dcm_file.StructureSetROISequence

                    # Extract the organ data (coordinates of all the slices) from the RTSTRUCT file
                    
                    organ_data={}
                    for j in range(len(contours)):
                        contour_data = np.array([])
                        #for loop to get the cordinate3 from all the slices of of the organ
                        for i in range(len(contours[j].ContourSequence)):
                            contour_data=np.concatenate((contour_data,(np.array(contours[j].ContourSequence[i].ContourData))))
                            #getting all the cordinate data to a data frame. with x,y,z cordinates
                            organ_data[f"organ{j}"]=pd.DataFrame(contour_data.reshape(int(len(contour_data)/3),3),columns=("X","Y","Z"))

        
                    organs_in_file = " ".join(organs[i].ROIName for i in range(len(contours)))
                    body_num = next((i for i, organ in enumerate(organs) if organ.ROIName == "Body" or "BODY"), None)
                    imported_organs = list(model_organs.keys())

                    #Modify organ_data to contain percentage x and y values
                    body_x_max = organ_data[f"organ{body_num}"]["X"].max()
                    body_x_min = organ_data[f"organ{body_num}"]["X"].min()
                    body_y_max = organ_data[f"organ{body_num}"]["Y"].max()
                    body_y_min = organ_data[f"organ{body_num}"]["Y"].min()
                    
                    body_x_axis = (body_x_max + body_x_min)/2
                    body_y_axis = (body_y_max + body_y_min)/2
                    
                    for i in range(len(organ_data)):
                        organ_data[f"organ{i}"]["X"] = (organ_data[f"organ{i}"]["X"]-body_x_axis)*100/body_x_max
                        organ_data[f"organ{i}"]["Y"] = (organ_data[f"organ{i}"]["Y"]-body_y_axis)*100/body_y_max
                    
                    
                    # Create an empty DataFrame
                    results = pd.DataFrame(columns=['organ_index', 'imported_organ', 'D_tot'])

                    for i in range(len(contours)):
                        if i != body_num:
                            for imported_organ in imported_organs:
                                D1, D12, D13 = directed_hausdorff(organ_data[f"organ{i}"][["X"]], model_organs[imported_organ][["X"]], seed=0)
                                #D2, D22, D23 = directed_hausdorff(organ_data[f"organ{i}"][["Y"]], model_organs[imported_organ][["Y"]], seed=0)
                                
                                D_tot = D1
                                # Create a temporary DataFrame for current iteration
                                temp_df = pd.DataFrame({'organ_index': [i], 'imported_organ': [imported_organ], 'D_tot': [D_tot]})
                                # Concatenate the temporary DataFrame with the main DataFrame
                                results = pd.concat([results, temp_df], ignore_index=True)

                    # Define a function to get the name of the closest imported organ for a given organ index
                    def get_closest_imported_organ_name(i):
                        if i in results['organ_index'].values:
                            filtered_results = results.loc[(results['organ_index'] == i) & (results['D_tot'].astype(float) <= 5)]
                            if len(filtered_results) > 0:
                                return filtered_results.loc[filtered_results['D_tot'].astype(float).idxmin(), 'imported_organ']
                            else:
                                return identify_PTV(i)
                        else:
                            return identify_PTV(i)

                    
                    #identifying CTV
                    def identify_CTV(x):   
                                              
                        for p in range(len(organs)):
                            organ_x1 = organ_data[f"organ{p}"]["X"][organ_data[f"organ{p}"]["Z"] == breast_z1]
                            organ_x2 = organ_data[f"organ{p}"]["X"][organ_data[f"organ{p}"]["Z"] == breast_z2]
                            organ_x3 = organ_data[f"organ{p}"]["X"][organ_data[f"organ{p}"]["Z"] == breast_z3]
                            
                            
                            x_diff_CTV = (body_x1.max()+body_x2.max()+body_x3.max()) - (organ_x1.max()+organ_x2.max()+organ_x3.max())
                            
                            x_diff[p] = x_diff_CTV
                            x_diff[body_num] = 1000
                            x_diff[PTV_organ_num] = 1000
                            

                        CTV_x_diff_min = min([value for value in x_diff.values() if not np.isnan(value)])
                        CTV_organ_num = list(x_diff.keys())[list(x_diff.values()).index(CTV_x_diff_min)]

                        if CTV_organ_num == x :
                            return 'CTV'
                        else:
                            return "---"
                    
                    x_diff = {}
                    #identifying PTV
                    def identify_PTV(x):
                        global breast_z1, breast_z2, breast_z3                      
                        global body_x1, body_x2, body_x3
                        
                        global PTV_organ_num  # Declare PTV_organ_num as a global variable
                        global PTV_x_diff_min
                        global Breast_R
                        
                        i = 0
                        for organ in organs:
                            if organ.ROIName == "Breast_R":
                                Breast_R = i
                            else:
                                i += 1

                        breast_z1 = organ_data[f"organ{Breast_R}"]["Z"].unique()[int(len(organ_data[f"organ{Breast_R}"]["Z"].unique()) / 3)]
                        breast_z2 = organ_data[f"organ{Breast_R}"]["Z"].unique()[int(len(organ_data[f"organ{Breast_R}"]["Z"].unique()) *2 / 3)]
                        breast_z3 = organ_data[f"organ{Breast_R}"]["Z"].unique()[int(len(organ_data[f"organ{Breast_R}"]["Z"].unique()) -1 )]
                        
                        body_x1 = organ_data[f"organ{body_num}"]["X"][organ_data[f"organ{body_num}"]["Z"] == breast_z1]
                        body_x2 = organ_data[f"organ{body_num}"]["X"][organ_data[f"organ{body_num}"]["Z"] == breast_z2]
                        body_x3 = organ_data[f"organ{body_num}"]["X"][organ_data[f"organ{body_num}"]["Z"] == breast_z3]
                        
                        for p in range(len(organs)):
                            organ_x1 = organ_data[f"organ{p}"]["X"][organ_data[f"organ{p}"]["Z"] == breast_z1]
                            organ_x2 = organ_data[f"organ{p}"]["X"][organ_data[f"organ{p}"]["Z"] == breast_z2]
                            organ_x3 = organ_data[f"organ{p}"]["X"][organ_data[f"organ{p}"]["Z"] == breast_z3]
                            
                            x_diff_PTV = (body_x1.max()+body_x2.max()+body_x3.max()) - (organ_x1.max()+organ_x2.max()+organ_x3.max())
                            
                            x_diff[p] = x_diff_PTV
                            x_diff[body_num] = 1000

                        PTV_x_diff_min = min([value for value in x_diff.values() if not np.isnan(value)])
                        PTV_organ_num = list(x_diff.keys())[list(x_diff.values()).index(PTV_x_diff_min)]
                        
                        if PTV_organ_num == x :
                            return 'PTV'                        
                        
                        else:
                            return identify_CTV(x)
                        
                    # Create a dataframe with the algorithm results for the current RTSTRUCT file
                    algorithm = pd.DataFrame({

                    "organ_number": range(len(organs)),
                    "organ_name": [organ.ROIName for organ in organs],
                    "organ_identified": [get_closest_imported_organ_name(i) for i in range(len(organs))],

                    })

                    algorithm["test"] = algorithm.apply(lambda row: "Pass" if ((row["organ_identified"] in row["organ_name"]) or (row["organ_name"] in row["organ_identified"]) ) else "Fail", axis=1)

                    if foldername not in result_dict:
                        result_dict[foldername] = []
                    result_dict[foldername].append(algorithm)
                    
                    def results():
                        for key, value in result_dict.items():
                            print(f"PATIENT {key}:")
                            print(value)
                            print()
                            
                    def result_stats():
                        result_stats = result_dict["1"][0][["organ_name", "test"]]

                        for i in range(2, len(result_dict) + 1):
                            result_stats = pd.concat([result_stats, result_dict[str(i)][0][["organ_name", "test"]]], ignore_index=True)

                        organ_value_counts = result_stats["organ_name"].value_counts()
                        organ_pass_counts = result_stats[result_stats["test"]=="Pass"]["organ_name"].value_counts()
                        pass_percentage = organ_pass_counts*100/organ_value_counts
                        
                        return pass_percentage.sort_values(ascending=False)

In [12]:
results()

PATIENT 1:
[   organ_number  organ_name organ_identified  test
0             0        Body              ---  Fail
1             1    Breast_R         Breast_R  Pass
2             2       Heart            Heart  Pass
3             3      Lung_L           Lung_L  Pass
4             4      Lung_R           Lung_R  Pass
5             5  PTV_InMarg              PTV  Pass]

PATIENT 10:
[   organ_number  organ_name organ_identified  test
0             0        Body              ---  Fail
1             1    Breast_R         Breast_R  Pass
2             2       Heart            Heart  Pass
3             3      Lung_L           Lung_L  Pass
4             4      Lung_R           Lung_R  Pass
5             5  PTV_InMarg              PTV  Pass]

PATIENT 100:
[   organ_number  organ_name organ_identified  test
0             0        Body              ---  Fail
1             1    Breast_R              ---  Fail
2             2       Heart              ---  Fail
3             3      Lung_L            

In [11]:
result_stats()

Lung_R        54.0
PTV_InMarg    53.0
Heart         40.0
Lung_L        37.0
Breast_R      12.0
BODY           NaN
Body           NaN
Name: organ_name, dtype: float64